### Config stuff

In [102]:
import random
import pyspark
from pyspark.sql import SparkSession, functions
import ConnectionConfig as cc
from pyspark.sql.functions import *
cc.setupEnvironment()
cc.listEnvironment()

HOMEBREW_PREFIX: /opt/homebrew
COMMAND_MODE: unix2003
INFOPATH: /opt/homebrew/share/info:
SHELL: /bin/zsh
PYTHONPATH: /Users/user/Desktop/data4_project_group5
__CFBundleIdentifier: com.jetbrains.pycharm
TMPDIR: /var/folders/k_/tkt88xx94n17f7_nvrzjrkwc0000gn/T/
LC_ALL: en_US.UTF-8
HOME: /Users/user
HOMEBREW_REPOSITORY: /opt/homebrew
PATH: /Users/user/Desktop/data4_project_group5/myenv/bin:/Users/user/Library/Java/JavaVirtualMachines/temurin-21.0.2/Contents/Home/bin:/opt/homebrew/opt/python@3.11/bin:/opt/homebrew/opt/python@3.11/bin:/opt/homebrew/bin:/opt/homebrew/opt/python@3.11/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Users/user/Desktop/KDG/VMware Fusion.app/Contents/Public:/usr/local/go/bin

In [103]:
spark = cc.startLocalCluster("dimUserInitial",4)
spark.getActiveSession()


# Creating the operational database
In order to run this demo you have to create a tutorial_op database and run the PostgreSQL_SalesOperational.sql script.


# Initial load
We will create a slowly changing dimension type 2 called dimSalesRep based on a sourceTable in our operational database called dbo.salesrep. SCD2  tables demand extra care and  because we will store hirstorical values of the dimension with the help of ranges.
This notebook will create the table and fill it with the initial data. A second notebook will be used for increments of new and changed data.

This is an example of the expected output (salesRepSK is different
```
+----------+-------------+-------------+-----------+-------------------+-------------------+--------------------+-------+
|salesRepID|         name|       office| salesRepSK|          scd_start|            scd_end|                 md5|current|
+----------+-------------+-------------+-----------+-------------------+-------------------+--------------------+-------+
|a46add1...|      Z. Jane|     New York|          0|1990-01-01 00:00:00|2100-12-12 00:00:00|303db545462092a92...|   true|
|s1fedf1...|   P. Chapman|       Berlin|          1|1990-01-01 00:00:00|2100-12-12 00:00:00|14b094c31bf9e4149...|   true|
|d5e6f77...|     T. Crane|     New York|          2|1990-01-01 00:00:00|2100-12-12 00:00:00|6c062f95defda9dc3...|   true|
```




In [108]:

# Set up JDBC connection
cc.set_connectionProfile("default")

df_users = spark.read \
    .format("jdbc") \
    .option("driver", cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "velo_users") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "userid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 20) \
    .load()

# Create a temporary view
df_users.createOrReplaceTempView("dimUser")
spark.sql("select * from dimUsers")

users_dim = spark.sql("select uuid() as userSK, *, to_timestamp('1999-01-01','yyyy-MM-dd') as scd_start, to_timestamp('2100-12-12','yyyy-MM-dd') as scd_end, md5(concat( street, number, zipcode, city, country_code)) as md5, True as current from dimUsers")

users_dim.printSchema()


users_dim.write.format("delta").mode("overwrite").saveAsTable("dimUser")

root
 |-- userSK: string (nullable = false)
 |-- userid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- street: string (nullable = true)
 |-- number: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- scd_start: timestamp (nullable = true)
 |-- scd_end: timestamp (nullable = true)
 |-- md5: string (nullable = true)
 |-- current: boolean (nullable = false)



## Delete the spark session

In [56]:
spark.stop()